In [ ]:
# Dependencies
!pip install gym==0.25.2
!pip install keras-rl2

In [ ]:
# Imports
import gym
import random
from ale_py import ALEInterface
from ale_py.roms import SpaceInvaders
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, InputLayer
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# **Environment**

In [ ]:
# Get ROM
ale = ALEInterface()
ale.loadROM(SpaceInvaders)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Env
env = gym.make("SpaceInvaders-v4")
h, w, c = env.observation_space.shape
actions = env.action_space.n

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [ ]:
# Env Test
episodes = 5
for episode in range(episodes):
  env.reset()
  done = False
  score = 0
  while not done:
    action = random.choice([0,1,2,3,4,5])
    n_states, reward, done, info = env.step(action)
    score+=reward
  print(f'Episode: {episode}, Score: {score}')

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


Episode: 0, Score: 80.0
Episode: 1, Score: 215.0
Episode: 2, Score: 485.0
Episode: 3, Score: 155.0
Episode: 4, Score: 215.0


# **Model**

In [ ]:
def build_model(height, width, channels, action):
  # Sequential Model
  model = Sequential()
  # Input Layer
  model.add(InputLayer(input_shape = (3,height, width, channels)))
  # Convolution Layer
  model.add(Conv2D(32, (8,8), strides= (4,4), activation = 'relu'))
  model.add(Conv2D(64, (4,4), strides= (2,2), activation = 'relu'))
  model.add(Conv2D(64, (3,3), activation = 'relu'))
  model.add(Flatten())
  # Dense Layer
  model.add(Dense(512, activation = 'relu'))
  model.add(Dense(256, activation = 'relu'))
  model.add(Dense(action, activation = 'linear'))
  # Return Model
  return model

In [ ]:
model = build_model(h, w, c, actions)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 3, 51, 39, 32)     6176      
                                                                 
 conv2d_4 (Conv2D)           (None, 3, 24, 18, 64)     32832     
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 22, 16, 64)     36928     
                                                                 
 flatten_1 (Flatten)         (None, 67584)             0         
                                                                 
 dense_3 (Dense)             (None, 512)               34603520  
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dense_5 (Dense)             (None, 6)                

# **Agent**

In [ ]:
# Agent
def build_agent(model, action):
  memory = SequentialMemory(limit = 1000, window_length = 3)
  policy  = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr = 'eps', value_max = 1., value_min = 0.1, value_test = 0.2, nb_steps = 10000)
  dqn = DQNAgent(model = model, policy = policy, memory = memory,
                 enable_dueling_network = True, dueling_type = 'avg',
                 nb_actions = action, nb_steps_warmup = 1000)
  return dqn

In [ ]:
agent = build_agent(model, actions)

# **Training**

In [ ]:
agent.compile(Adam(learning_rate = 1e-4))
agent.fit(env, nb_steps = 10000, visualize = False, verbose = 2)

Training for 10000 steps ...
  502/10000: episode: 1, duration: 23.955s, episode steps: 502, steps per second:  21, episode reward: 110.000, mean reward:  0.219 [ 0.000, 30.000], mean action: 2.673 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --


In [ ]:
scores = agent.test(env, nb_episodes = 2, visualize = False)
print(np.mean(scores.history['episode_reward']))